In [1]:
import os
from pathlib import Path
import multiprocessing as mp

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from skimage.data import imread
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

camera_model.pkl
test1
train



In [2]:
input_path = Path('/home/mashrin/input')
print(input_path)
train_path = input_path/'train' # /home/mashrin/input/train
test_path = input_path/'test1'
print(test_path)
#os.listdir(test_path)
os.listdir(train_path)
#os.walk("test_path")

/home/mashrin/input
/home/mashrin/input/test1


['HTC-1-M7',
 'Motorola-Nexus-6',
 'Sony-NEX-7',
 'Samsung-Galaxy-S4',
 'iPhone-6',
 'iPhone-4s',
 'Motorola-X',
 'Motorola-Droid-Maxx',
 'Samsung-Galaxy-Note3',
 'LG-Nexus-5x']

In [3]:
cameras = os.listdir(train_path)

train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(train_path / camera)):
        train_images.append((camera, fname))

train = pd.DataFrame(train_images, columns=['camera', 'fname'])
print(train.shape)
train.sample(5)

(2750, 2)


,camera,fname
68,HTC-1-M7,(HTC-1-M7)160.jpg
1155,iPhone-6,(iP6)149.jpg
1622,iPhone-4s,(iP4s)74.jpg
447,Motorola-Nexus-6,(MotoNex6)254.jpg
358,Motorola-Nexus-6,(MotoNex6)174.jpg


In [4]:
test_images = []
for fname in sorted(os.listdir(test_path)):
    test_images.append(fname)
    #print(test_images)
test = pd.DataFrame(test_images, columns=['fname'])
print(test.shape)
test.head(5)

(70, 1)


,fname
0,(GalaxyN3)22.jpg
1,(GalaxyN3)23.jpg
2,(GalaxyN3)24.jpg
3,(GalaxyN3)25.jpg
4,(GalaxyS4)11.jpg


In [5]:
def color_stats(q, iolock):
    
    while True:
        
        img_path = q.get()
        if img_path is None:
            break
            
        if type(img_path) is tuple:
            img = imread(train_path / img_path[0] / img_path[1])
            key = img_path[1]
        else:
            img = imread(test_path / img_path)
            key = img_path

        # Some images read return info in a 2nd dim. We only want the first dim.
        if img.shape == (2,):
            img = img[0]

        color_info[key] = (img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean(),
                           img[:, :, 0].std(),  img[:, :, 1].std(),  img[:, :, 2].std())    

In [6]:
cols = ['a0', 'a1', 'a2', 's0', 's1', 's2']

for col in cols:
    train[col] = None
    test[col] = None

In [7]:
NCORE = 8

color_info = mp.Manager().dict()

# Using a queue since the image read is a bottleneck
q = mp.Queue(maxsize=NCORE)
iolock = mp.Lock()
pool = mp.Pool(NCORE, initializer=color_stats, initargs=(q, iolock))

for i in train_images:
    q.put(i)  # blocks until q below its max size

for i in test_images:
    q.put(i)  # blocks until q below its max size
    
# tell workers we're done
for _ in range(NCORE):  
    q.put(None)
pool.close()
pool.join()

color_info = dict(color_info)

In [8]:
for n, col in enumerate(cols):
    train[col] = train['fname'].apply(lambda x: color_info[x][n])
    test[col] = test['fname'].apply(lambda x: color_info[x][n])

In [9]:
train.sample(5)
test.sample(5)

,fname,a0,a1,a2,s0,s1,s2
58,(iP4s)23.jpg,100.393921,107.554872,91.045082,64.098100,66.410812,76.357379
44,(MotoX)14.jpg,64.859726,60.598672,54.532078,49.729233,53.594673,48.027553
52,(Nex7)272.JPG,111.873796,111.478324,117.324384,88.888926,89.811109,90.412447
63,(iP4s)28.jpg,114.706468,120.588007,126.645102,45.167838,45.812141,49.952080
59,(iP4s)24.jpg,78.493214,88.646666,79.019691,58.903285,69.635095,82.740934


In [10]:
y = train['camera'].values
X_train = train[cols].values
X_test = test[cols].values

In [11]:
clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
y_pred = clf.predict(X_test)
#clueless = pd.read_csv(input_path / 'sample_submission.csv', index_col='fname')
#clueless['camera'] = y_pred
#clueless.to_csv('clueless.csv')
#df = pd.read_csv('clueless.csv')
#df
#print(X_test)
print(y_pred)

['Samsung-Galaxy-Note3' 'Samsung-Galaxy-Note3' 'Samsung-Galaxy-Note3'
 'Samsung-Galaxy-Note3' 'Samsung-Galaxy-S4' 'Samsung-Galaxy-S4'
 'Samsung-Galaxy-S4' 'Samsung-Galaxy-S4' 'Samsung-Galaxy-S4'
 'Samsung-Galaxy-S4' 'Samsung-Galaxy-S4' 'Samsung-Galaxy-S4'
 'Samsung-Galaxy-S4' 'HTC-1-M7' 'HTC-1-M7' 'HTC-1-M7' 'HTC-1-M7' 'HTC-1-M7'
 'HTC-1-M7' 'HTC-1-M7' 'HTC-1-M7' 'HTC-1-M7' 'HTC-1-M7' 'HTC-1-M7'
 'HTC-1-M7' 'LG-Nexus-5x' 'LG-Nexus-5x' 'LG-Nexus-5x' 'LG-Nexus-5x'
 'LG-Nexus-5x' 'LG-Nexus-5x' 'LG-Nexus-5x' 'LG-Nexus-5x'
 'Motorola-Droid-Maxx' 'Motorola-Droid-Maxx' 'Motorola-Droid-Maxx'
 'Motorola-Droid-Maxx' 'Motorola-Droid-Maxx' 'Motorola-Nexus-6'
 'Motorola-Nexus-6' 'Motorola-Nexus-6' 'Motorola-X' 'Motorola-X'
 'Motorola-X' 'Motorola-X' 'Sony-NEX-7' 'Sony-NEX-7' 'Sony-NEX-7'
 'Sony-NEX-7' 'Sony-NEX-7' 'Sony-NEX-7' 'Sony-NEX-7' 'Sony-NEX-7'
 'Sony-NEX-7' 'Sony-NEX-7' 'Sony-NEX-7' 'iPhone-4s' 'iPhone-4s' 'iPhone-4s'
 'iPhone-4s' 'iPhone-4s' 'iPhone-4s' 'iPhone-4s' 'iPhone-4s' 'iPhone-4s'

In [14]:
import pickle

In [16]:
filename = '/home/mashrin/input/camera_model.pkl'

pickle.dump(clf, open(filename, 'wb'))

#pickle.dump(file=Saveopen(filename,"w"),obj=clf)